This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/pend.html).  I recommend having both windows open, side-by-side!


In [1]:
import datetime

from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from matplotlib.transforms import Affine2D
import matplotlib.pyplot as plt

import numpy as np

from environmental_model import EnvironmentalModel

trying to make sailboat model chaotically

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from pydrake.all import (
    AbstractValue,
    Cylinder,
    DiagramBuilder,
    DirectCollocation,
    FiniteHorizonLinearQuadraticRegulatorOptions,
    FramePoseVector,
    LeafSystem,
    LeafSystem_,
    LogVectorOutput,
    MakeFiniteHorizonLinearQuadraticRegulator,
    MathematicalProgram,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    Rgba,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Solve,
    SnoptSolver,
    StartMeshcat,
    TemplateSystem,
    TrajectorySource,
    namedview,
)
#from underactuated import ConfigureParser
#from underactuated.scenarios import AddShape

In [3]:
import pydrake 

class SailboatGeometry(LeafSystem):
    def __init__(self): #probs some other stuff here too? 
        LeafSystem.__init__(self)
        mbp = MultibodyPlant(1.0) 
        #parser = Parser(mbp, scene_graph)
        
#sail = pydrake.multibody.plant.Wing()


### Polar Plot

Below function imports a Melges 24 polar plot in order to calculate the maximum speed that can be achieved. Currently it just goes to the closest possible value that exists to the inputs. Goal is to have a continuous polar plot to draw upon later/linearly interpolate between them in a bit. 

In [4]:
import csv
import pandas as pd
polar_plot = pd.read_csv('melges_polar.csv', sep=';')
#print(polar_plot.iloc[:,0])
'''
first list in list is wind velocities
index 0 is the angle at which you're headed
rest of them are the speeds you can achieve, assume we auto reach that new speed
use .iloc to splice the columns
'''

def print_polar_plot_data():
    cols = [6, 8, 10, 12, 14, 16, 20]
    rows = [0.0, 38.2, 38.5, 38.8, 39.3, 40.5, 43.0, 52.0, 60.0, 75.0, 90.0, 110.0, 120.0, 135.0, 142.6, 143.3, 144.7, 145.6, 147.8, 148.3, 149.6, 150.0]

    for ri, r in enumerate(rows):
        for ci, c in enumerate(cols):
            data = polar_plot.iloc[ri+1, ci+1]

            if data != 0:
                print(f'{r}, {c}, {data}, 1')

# print_polar_plot_data()


def mps_to_knots(mps):
    '''
    Converts m/s to knots
    '''
    return mps * 1.94384449


def boat_vel_max(heading, wind_velocity):
    '''
    I used this website to fit a 2D surface to the polar plot data
    http://www.findcurves.com/Equation/3/Polynomial/User-Selectable%20Polynomial/

    This provides a closed form approximation of the polar plot which
    lets the mathematical program do its thing when solving the
    optimization

    @param state: the sailboat state vector
    @param wind_vector: the current wind vector np.array([V_x, V_y])
    @returns: the maximum velocity (as a scalar) the boat can attain with the given state and wind_vector
    '''

    # if not 38 <= heading <= 150:
    #     return 0

    x_in = heading
    y_in = wind_velocity

    temp = 0.0

    # coefficients
    a = -4.3090504667975598E+00
    b = 1.3706700256696367E+00
    c = -8.3202164989653701E-02
    d = 1.8463302630606845E-03
    f = 1.6370859651215258E-01
    g = 9.9951004621403445E-04
    h = -7.1233272459415506E-04
    i = 1.3913446797314238E-05
    j = -9.3528258078313096E-04
    k = -2.5514917241328935E-04
    m = 3.4724643668681521E-05
    n = -8.2856020551508026E-07
    p = -2.7065066833418202E-07
    q = 1.7042158865865885E-06
    r = -2.0666643065103133E-07
    s = 5.3326758086124256E-09

    temp += a
    temp += b * y_in
    temp += c * np.power(y_in, 2.0)
    temp += d * np.power(y_in, 3.0)
    temp += f * x_in
    temp += g * x_in * y_in
    temp += h * x_in * np.power(y_in, 2.0)
    temp += i * x_in * np.power(y_in, 3.0)
    temp += j * np.power(x_in, 2.0)
    temp += k * np.power(x_in, 2.0) * y_in
    temp += m * np.power(x_in, 2.0) * np.power(y_in, 2.0)
    temp += n * np.power(x_in, 2.0) * np.power(y_in, 3.0)
    temp += p * np.power(x_in, 3.0)
    temp += q * np.power(x_in, 3.0) * y_in
    temp += r * np.power(x_in, 3.0) * np.power(y_in, 2.0)
    temp += s * np.power(x_in, 3.0) * np.power(y_in, 3.0)
    return temp


print(boat_vel_max(43.0, 16))  # 43.0, 16, 6.97
print(boat_vel_max(90.0, 8))  # 90.0, 8, 6.77

# def closest_value(list_a, k):
#   return list_a[min(range(len(list_a)), key=lambda i: abs(list_a[i]-k))]

# def boat_vel_max(heading, wind_velocity):
#     '''
#     Looks up the maximum velocity from a polar plot for <some boat>, no linear interpolation for now
#     assumes the wind_velocity lines up with whatever is in the polar plot

#     Edit once it gets better, add linear interpolation in

#     @param state: the sailboat state vector
#     @param wind_vector: the current wind vector np.array([V_x, V_y])
#     @returns: the maximum velocity (as a scalar) the boat can attain with the given state and wind_vector
#     '''
#     #which column to look at for max speed
#     wind_vels = polar_plot.columns[1:].tolist() #polar_plot.iloc[0,:]
#     wind_vels = [float(i) for i in wind_vels]
#     #print(wind_vels)
#     wind_column = closest_value(wind_vels,wind_velocity)
#     wind_column_idx = wind_vels.index(wind_column)

#     #heading column to find the max speed, also checking that boat is not in irons
#     #if heading < polar_plot.iloc[1,0]:
#     #    boat_velocity = 0
    
#     #figure out max speed
#     #else:
#     headings = polar_plot.iloc[:,0].tolist()
#     #print(headings)
#     heading_idx = headings.index(closest_value(headings,heading))
    
#     boat_velocity = polar_plot.iloc[heading_idx,wind_column_idx+1]
#     return boat_velocity

# # print(boat_vel_max(52,6))

6.781365146288449
6.813667749958175


### Dynamics of Sailboat with Drag and Lift

working on getting the dynamics of a sailboat working such that it doesn't exist at max speed constantly

coding equations of motion here

In [5]:
def get_Re(rho, V, l, mu):
    Re = rho * V * l / mu
    return Re

In [6]:
def log10_approx(x):
    '''
    Taylor expansion of log10(x) to make the MathematicalProgram happy
    '''
    ln10 = np.log(10)
    val = ((x-1)/ln10) - ((x-1)**2/(2*ln10)) + ((x-1)**3/(3*ln10))
    return val

def unit_vector(vector):
    """
    Returns the unit vector of the vector.
    https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python
    """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
        https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python
    """
    print(f'v1:{v1}\tv2: {v2}')
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.dot(v1_u, v2_u))
    # return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



In [7]:
def get_boatspeed(self, pos):
        #v = np.sqrt(pos[4]**2 + pos[5]**2) #boat speed
        v = pos[4]
        return v

In [8]:
class rudder():
    def __init__(self, span, chord, rho, mu):
        '''
        dimensions for rudder
        physical properties of water
        '''
        self.b = span 
        self.c = chord
        self.S = self.b*self.c
        self.AR = self.b/self.c
        self.rho = rho
        self.mu = mu

    def find_cl(self, AoA):
        cl = 2*np.pi*AoA
        return cl

    def find_lift(self, AoA, v): 
        #finding coefficient of lift, based on sail angle of attack, finds lift using that
        '''
        AoA = angle of attack
        rho = density of fluid we are moving through
        v = velocity of foil
        S = sail area
        '''

        cl = self.find_cl(AoA)
        L = .5 * self.rho * (v**2) * cl * self.S
        return L
    
    def find_drag(self, AoA, v):
        '''
        thin air foil, can assume flat plate when trimmed, using blasius to find drag
        assume laminar flow, FVA 4.102
        Dp = profile drag
        Di = induced drag
        '''
        
        cl = self.find_cl(AoA)
        Re_l = get_Re(self.rho, v, self.c, self.mu)
        S_wet = 2 * self.S
        Cf = 1.328 / Re_l**.5
        Dp = .5 * self.rho * (v**2) * S_wet * Cf
        Di = .5 * self.rho * (v**2) * self.S * (cl**2 / (np.pi * self.AR))
        D = Dp + Di
        return D


In [9]:
#
# Sailboat State Vector Format:
#
# [
#   sail_angle: the angle in degrees of the sail relative to the hull measured counterclockwise from the y axis
#   x: the x position of the center of the hull in meters
#   y: the y position of the center of the hull in meters
#   heading: the hull's heading in degrees measured clockwise from the positive y axis
#   x_dot: time derivative of x
#   y_dot: time derivative of y
# ]
#


#
# Sailboat Control Vector Format:
#
# Assume that we can update the sail angle and heading instantaneously
#
# [
#   sail_angle: the angle in radians of the sail relative to the hull measured counterclockwise from the forward axis
#   heading: the hull's heading in degrees measured clockwise from the positive y axis
# ]
#


class SailboatDynamicsModel():

    # Sizes of the state and control vectors
    STATE_SIZE = 6
    CONTROL_SIZE = 2
    
    def __init__(self, sail_area, boat_mass, boat_length, environmental_model):
        '''
        initializing most of the variables to use for finding the dynamics of overall boat
        Melges 24 sail area usually 17.1871 m^2. 
        '''
        self.sail_area = sail_area
        self.mass = boat_mass
        self.length = boat_length
        self.cf = 0 #skin friction is 0 at time 0

        self.rho_air = 1.225
        self.mu_air = 1.825 * (10**(-5))

        self.add_rudder()

        self.environment = environmental_model


    def get_boatspeed(self, pos):
        v = np.sqrt(pos[4]**2 + pos[5]**2) + .01 #boat speed
        return v
    

    def add_rudder(self):
        '''
        adds rudder to the boat, need this to account for drag
        '''
        self.span = 1.2 #rough dimensions of melges 24
        self.chord = .48
        self.rho_water = 1000
        self.mu_water = 8.9 * 10**(-4)
        self.rudder = rudder(self.span, self.chord, self.rho_water, self.mu_water)

    def relative_wind_angle(self, state, wind_vector):
        '''
        Returns the angle (in degrees) between the wind_vector and the current heading
        '''

        apparent_wind_vector = wind_vector - state[-2:]  # TODO: incorporate this into the rest of the code

        heading_rad = np.pi * state[3] / 180
        heading_vector = np.array([np.sin(heading_rad), np.cos(heading_rad)])

        # print(f'state[3]: {state[3]}')
        # print(f'heading_vector: {heading_vector}')
        # print(f'-wind_vector: {-wind_vector}')

        relative_wind_angle_rad = angle_between(heading_vector, -wind_vector)
        relative_wind_angle_deg = 180 * relative_wind_angle_rad / np.pi

        return relative_wind_angle_deg


    def get_apparent_wind(self, pos, wind):
        '''
        find direction of wind the boat is perceiving while in motion
        x = state vector
        wind = wind angle and heading
        https://www.onemetre.net//Design/AppWind/appwind.htm
        '''
        TWA = wind[0] #true wind angle
        TWS = wind[1] #true wind speed
        v = self.get_boatspeed(pos) #boat speed
        appWindAngle = np.arctan(np.sin(TWA)*TWS)/(v+np.cos(TWA)*TWS)
        appWindSpeed = np.sqrt((np.sin(TWA)*TWS)**2+(v+np.cos(TWA)*TWS)**2)
        return appWindAngle, appWindSpeed


    def sail_drag(self,pos,wind):
        '''
        add a sail, using air properties and sail sizes
        similar class to rudder, just takes sail area instead of span and chord 
        '''
        #3.9 is avg chord of melges 24 sail, using this for now
        theta, v = self.get_apparent_wind(pos,wind)
        Cf_sail = 1.328 / get_Re(self.rho_air, v+.01 ,3.9 , self.mu_air)**.5 #Reynolds for air
        Dp = .5 * self.rho_air * (v**2) * self.sail_area * Cf_sail

        cl = 1.6 #about the lift of a traditional rig
        AR = 8.8/3.9
        Di = .5 * self.rho_air * (v**2) * self.sail_area * (cl**2 / (np.pi * AR))

        D = (Dp + Di) * np.cos(theta) #this gets the force in line with the direction the boat is travelling
        return D


    def hull_drag(self, pos):
        '''
        drag of the actual boat in the water 
        https://www.usna.edu/NAOE/_files/documents/Courses/EN400/02.07%20Chapter%207.pdf
        '''
        nu = self.mu_water/self.rho_water
        Rn = self.length * self.get_boatspeed(pos)/ nu
        denom = (log10_approx(Rn)-2)**2 + .01
        Cf = .075 / denom #(self.get_boatspeed(pos)+.01)#
        D = .5 * self.rho_water * self.get_boatspeed(pos)**2 * self.length #not sure if i need a wetted surface area here
        return D


    def get_total_drag(self, pos, wind):
        '''
        combines rudder, hull, and sail drag
        this drag all is slowing boat down in same opposite direction it is travelling
        we are assuming the keel perfectly cancels out side slip
        '''
        angle = self.relative_wind_angle(pos, wind)
        #D_rudder = self.rudder.find_drag(angle, self.get_boatspeed(pos))  # TODO: Added AoA to find_drag because it needs cl
        D_hull = self.hull_drag(pos)
        D_sail = self.sail_drag(pos, wind)
        total_drag =  D_hull + D_sail #D_rudder +
        return total_drag
    

    def get_forward_force(self, pos, wind):
        '''
        finding portion of lift that drives the boat forward from sail and rudder
        also does calculation for main sail lift here, sail lift will be based on apparent wind
        '''
        #rudder lift
        lift_rudder = self.rudder.find_lift(pos[3], self.get_boatspeed(pos))

        cl = 2 #rough cl of rig moving faster(apparent wind makes speed faster)
        theta, v = self.get_apparent_wind(pos,wind)
        v = .5
        lift_sail = .5 * self.rho_air * (v**2) * self.sail_area * cl
        L = lift_rudder + lift_sail
        return L


    def continuous_dynamics(self, pos, wind):
        '''
        combine the dynamics, returns acceleration vector for now
        '''
        # thrust acceleration
        F_lift = self.get_forward_force(pos,wind)
        F_drag = self.get_total_drag(pos,wind)
        force = F_lift - F_drag
        a = force / self.mass
        return a


    def discrete_dynamics(self, pos, control, time, time_step=0.01, debug=False):
        '''
        @param state: the current sailboat state vector
        @param control: the current sailboat control vector
        @param time: the current time in seconds
        @param time_step: the size of the time step to take in seconds
        @returns: the new state after time_step seconds given the current state and control vectors
        '''

        assert pos.shape == (self.STATE_SIZE,)
        assert control.shape == (self.CONTROL_SIZE,)

        env_conditions = self.environment.get_conditions(time)
        wind_vector = env_conditions['wind'](pos[1:3])

        acceleration = self.continuous_dynamics(pos, wind_vector)
        print(pos[4])

        # if debug:
        #     print(f'DYNAMICS:')
        #     print(f'\tstate: {state}')
        #     print(f'\tcontrol: {control}')
        #     print(f'\twind_vector: {wind_vector}')
        #     print(f'\twind_vector_knots: {mps_to_knots(wind_vector)}')
        #     # print(f'\twind_angle_rad: {wind_angle_rad}')
        #     print(f'\twind_angle_deg: {wind_angle_deg}')
        #     print(f'\theading: {state[3]}')
        #     # print(f'\trelative_wind_angle_deg: {relative_wind_angle_deg}')
        #     print(f'\tmax_v: {max_v}')

        # Assume that the boat only travels forward/backward
        # Assume that the hull heading can change instantaneously
        # Assume that the sail angle can change instantaneously

        heading_rad = np.pi * pos[3] / 180
        new_v_x = pos[4] - acceleration * np.sin(heading_rad)*time_step
        new_v_y = pos[5] + acceleration * np.cos(heading_rad)*time_step

        new_state = np.array([
            control[0],
            pos[1] + time_step * pos[4],
            pos[2] + time_step * pos[5],
            control[1],
            new_v_x,
            new_v_y
        ])

        return new_state
    

### Trajectory Simulator

In [10]:
def calculate_trajectory(initial_state, dynamics_model, control_input, time_step=0.1):
    '''
    @param initial_state: a numpy array with the sailboat starting conditions
    @param dynamics_model: an instance of the SailboatDynamicsModel class
    @param control_input: a (CONTROL_SIZE, N) numpy array with the controls through the 
    @param time_step: the time step in seconds
    @returns
    '''
    trajectory = np.zeros((control_input.shape[0] + 1,dynamics_model.STATE_SIZE))
    trajectory[0] = initial_state

    env_conditions = [dynamics_model.environment.get_conditions(0)]

    for i, u in enumerate(control_input):
        trajectory[i+1] = dynamics_model.discrete_dynamics(trajectory[i], u, time_step*i, time_step)
        env_conditions.append(dynamics_model.environment.get_conditions(time_step * i))

    return trajectory, env_conditions

### Trajectory Optimization

In [11]:
def calculate_control(initial_state, goal_state, dynamics_model, num_time_steps, time_step=0.1):
    '''
    @param initial_state: a numpy array with the sailboat starting conditions
    @param goal_state: a numpy array with the desired sailboat final conditions
    @param dynamics_model: an instance of the SailboatDynamicsModel class
    @param num_time_steps: the number of time steps to include in the control output
    @param time_step: the time step in seconds
    @returns: if an optimal trajectory could be found, return a
    (num_time_steps, dynamics_model.CONTROL_SIZE) array of the control output that
    will direct the boat from the initial_state to the goal state. Otherwise, return None
    '''
    assert initial_state.shape == (dynamics_model.STATE_SIZE,)
    assert goal_state.shape == (dynamics_model.STATE_SIZE,)


    prog = MathematicalProgram()

    state = prog.NewContinuousVariables(num_time_steps + 1, dynamics_model.STATE_SIZE, "state")
    control = prog.NewContinuousVariables(num_time_steps, dynamics_model.CONTROL_SIZE, "control")

    # Apply dynamics constraints
    for t in range(num_time_steps):
        next_state = dynamics_model.discrete_dynamics(
            state[t,:], control[t], t * time_step, time_step
        )

        residuals = next_state - state[t+1]

        for residual in residuals:
            prog.AddConstraint(residual == 0)

        # The heading should be between -180 and 180 degrees
        # The polar diagram is approximated with a polynomial, so this
        # prevents it from plugging in a large heading and getting a
        # crazy value
        prog.AddConstraint(state[t][3] <= 180)
        prog.AddConstraint(-180 <= state[t][3])

    # Add constraint that the first state should be the initial state
    for i in range(dynamics_model.STATE_SIZE):
        prog.AddConstraint(state[0][i] == initial_state[i]) 

    # Add constraint that final state should be close to the goal state
    for i in {1, 2}:
        # x and y positions should be close to the same, but the rest don't matter
        prog.AddConstraint(state[-1][i] - goal_state[i] <= 1) 
        prog.AddConstraint(-1 <= state[-1][i] - goal_state[i])

    def gen_cost():
        '''
        Cost function that penalizes adjacent control commands being
        different from each other. Should hopefully make it go in
        straight lines
        '''
        cost = 0
        for t in range(1, num_time_steps):
            cost += (control[t][1] - control[t-1][1])**2
        return cost

    # Add cost function
    prog.AddCost(gen_cost())

    # solve mathematical program
    solver = SnoptSolver()
    result = solver.Solve(prog)
    
    if result.is_success():
        control_opt = result.GetSolution(control)
        print(f'solution state: {result.GetSolution(state)}')
        return control_opt
    
    return None


In [12]:
def line_search(initial_state, goal_state, dynamics_model, max_time_steps=100, time_step=0.1):

    # Dumb search rn
    # TODO: make this binary search

    for num_time_steps in range(5, max_time_steps):
        result = calculate_control(initial_state, goal_state, dynamics_model, num_time_steps, time_step)

        if result is not None:
            return result

    return None

In [13]:
def draw_frame(frame_args):
    '''
    @param frame_args: a tuple of (
        state: the sailboat state vector,
        environment_conditions: the current environmental_conditions
    )
    '''

    state, environment_conditions = frame_args

    PLOT_SIZE = 200

    plt.cla()
    axes = plt.gca()
    axes.set_xlim(-PLOT_SIZE, PLOT_SIZE)
    axes.set_ylim(-PLOT_SIZE, PLOT_SIZE)

    #######################
    # Draw sailboat
    #######################

    BOAT_WIDTH = 9
    BOAT_LENGTH = 21
    SAIL_WIDTH = 3
    SAIL_HEIGHT = 9

    hull_transform = Affine2D().rotate_deg_around(*state[1:3], -state[3]) + axes.transData
    hull = Rectangle(
        (
            state[1] - (BOAT_WIDTH / 2),
            state[2] - (BOAT_LENGTH / 2)
        ),
        BOAT_WIDTH,
        BOAT_LENGTH,
        color='#3399ff',
        transform=hull_transform
    )
    axes.add_patch(hull)

    sail_transform = Affine2D().rotate_deg_around(*state[1:3], state[0] - state[3]) + axes.transData
    sail = Rectangle(
        (state[1], state[2]),
        SAIL_WIDTH,
        SAIL_HEIGHT,
        angle=state[0],
        color='#ac7339',
        transform=sail_transform
    )
    axes.add_patch(sail)

    #######################
    # Draw wind vectors
    #######################
    
    # GRID_SIZE = 15
    
    # xs, ys = np.meshgrid(
    #     np.linspace(-PLOT_SIZE, PLOT_SIZE, GRID_SIZE),
    #     np.linspace(-PLOT_SIZE, PLOT_SIZE, GRID_SIZE),
    # )

    # xs = xs.flatten()
    # ys = ys.flatten()

    # for x, y in zip(xs, ys):
    #     wind_vector = environment_conditions['wind']((x, y))
    #     axes.quiver(
    #         x, y, 
    #         *wind_vector,
    #         headwidth=2, headlength=1, headaxislength=1,
    #         color='#009900',

    #     )



def animate_trajectory(trajectory, environment_conditions, save_timestamp=False):
    '''
    @param trajectory: a series of state vectors over time as a TxN np.array
    @param environment_conditions: a series of the environmental conditions over time
    @param save_timestamp: if True, include the current timestamp in the filename
    '''
    
    fig = plt.figure(figsize=(6,4))
    axes = fig.add_subplot(1,1,1)
    plt.title("Sailboat Trajectory")
    plt.gca().set_aspect('equal')

    anim = FuncAnimation(
        fig, draw_frame,
        frames=zip(trajectory, environment_conditions),
        interval=250,  # time between frames in ms
    )

    if save_timestamp:
        timestamp = '{:%m-%d_%H-%M-%S}'.format(datetime.datetime.now())
        filename = f'trajectory_animation_{timestamp}.gif'
    else:
        filename = 'trajectory_animation.gif'

    anim.save('animations/' + filename)


In [14]:
initial_state = np.array([
    5, 1, 2, 45, 0.1, 0.1,
])
goal_state = np.array([
    20, 3, 10, -90, 0.5, 0.5,
])
dynamics_model = SailboatDynamicsModel(17.1871, 808, 7.31, EnvironmentalModel())
time_step=1

optimal_control = line_search(initial_state, goal_state, dynamics_model, 100, time_step)
print(f'control_output:\n{optimal_control}')

trajectory, env_conditions = calculate_trajectory(initial_state, dynamics_model, optimal_control, time_step)
print(f'trajectory:\n{trajectory}')

animate_trajectory(trajectory, env_conditions)

# NUM_STEPS = 10

# trajectory = np.zeros((NUM_STEPS, 6))
# trajectory[:, 0] = 10 * np.array(range(NUM_STEPS))
# trajectory[:, 1] = 10 * np.array(range(NUM_STEPS))
# trajectory[:, 3] = 10 * np.array(range(NUM_STEPS))

# env = EnvironmentalModel()
# environment_conditions = [env.get_conditions(t) for t in np.linspace(0, 2, NUM_STEPS)]

# print(trajectory)
# print(environment_conditions)

# animate_trajectory(trajectory, environment_conditions)

v1:[<Expression "sin(((3.1415926535897931 * state(0,3)) / 180))">
 <Expression "cos(((3.1415926535897931 * state(0,3)) / 180))">]	v2: [-1 -1]
state(0,4)
v1:[<Expression "sin(((3.1415926535897931 * state(1,3)) / 180))">
 <Expression "cos(((3.1415926535897931 * state(1,3)) / 180))">]	v2: [-1 -1]
state(1,4)
v1:[<Expression "sin(((3.1415926535897931 * state(2,3)) / 180))">
 <Expression "cos(((3.1415926535897931 * state(2,3)) / 180))">]	v2: [-1 -1]
state(2,4)
v1:[<Expression "sin(((3.1415926535897931 * state(3,3)) / 180))">
 <Expression "cos(((3.1415926535897931 * state(3,3)) / 180))">]	v2: [-1 -1]
state(3,4)
v1:[<Expression "sin(((3.1415926535897931 * state(4,3)) / 180))">
 <Expression "cos(((3.1415926535897931 * state(4,3)) / 180))">]	v2: [-1 -1]
state(4,4)


RuntimeError: Exception while evaluating SNOPT costs and constraints: 'Division by zero: 0 / 0(0 / (2 * sqrt((pow(state(1,4), 2) + pow(state(1,5), 2)))))
'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8d85514d-cf5d-478e-b379-38f04f9dde27' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>